In [15]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm

In [16]:
class HandwritingDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None,label_encoder=None): #sets the directories 
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform
        self.label_encoder = label_encoder

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]  # filename
        label_str = self.data.iloc[idx, 1]  # identity (e.g., "ALICE")
        img_path = os.path.join(self.img_dir, img_name)
        image = Image.open(img_path).convert("L")  # grayscale

        if self.transform:
            image = self.transform(image)

        if self.label_encoder:
            label = self.label_encoder.transform([label_str])[0]
        else:
            label = label_str

        return image, label


In [17]:
transform = transforms.Compose([
    transforms.Resize((64, 256)),  # resize all to 64x256
    transforms.ToTensor(),         # scales px values to [0, 1] instead of [0, 255]
    transforms.Normalize((0.5,), (0.5,))  # Normalize grayscale images to [-1, 1]
])

In [18]:
#encode names to numerical classes

all_labels = pd.concat([
    pd.read_csv('csv/written_name_train_v2.csv')["IDENTITY"],
    pd.read_csv('csv/written_name_validation_v2.csv')["IDENTITY"],
    pd.read_csv("csv/written_name_test_v2.csv")["IDENTITY"] 
])

le = LabelEncoder()

le = LabelEncoder()
le.fit(all_labels)

LabelEncoder()

In [19]:
train_dataset = HandwritingDataset(
    csv_file='csv/written_name_train_v2.csv',
    img_dir='dataset/train_v2/train',
    transform=transform,
    label_encoder=le
)

val_dataset = HandwritingDataset(
    csv_file='csv/written_name_validation_v2.csv',
    img_dir='dataset/validation_v2/validation',
    transform=transform
)

test_dataset = HandwritingDataset(
    csv_file='csv/written_name_test_v2.csv',
    img_dir='dataset/test_v2/test',
    transform=transform
)

In [20]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)

In [21]:
class useCNN(nn.Module):
    def __init__(self,num_classes):
        super(useCNN, self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=3)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(32,64,kernel_size=3)
        self.fc1 = nn.Linear(64 * 14 * 62, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 32, 31, 127]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 64, 14, 62]
        x = x.view(x.size(0), -1)             # [B, 55424]
        x = F.relu(self.fc1(x))               # [B, 256]
        x = self.fc2(x)    # [B, num_classes]
        return x


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = useCNN(num_classes=len(le.classes_)).to(device)

In [23]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
checkpoint_path = "checkpoint.pth"
start_epoch = 0
num_epochs = 5

In [25]:
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming from epoch {start_epoch}")
else:
    print("Starting fresh training...")

Starting fresh training...


In [ ]:
for epoch in range(start_epoch, num_epochs):
    model.train()
    total_loss = 0

    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for images, labels in progress_bar:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({"Batch Loss": loss.item()})

    print(f"✅ Epoch {epoch+1} complete | Total Loss: {total_loss:.4f}")

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss
    }, checkpoint_path)
    print("💾 Checkpoint saved.\n")

KeyboardInterrupt: 

In [ ]:
model.eval()
correct = 0
total = 0

In [ ]:
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

In [ ]:
print(f"Validation Accuracy: {100 * correct / total:.2f}%")